In [ ]:
# 1. Install dan import library
!pip install pandas scikit-learn tensorflow

In [ ]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer
from tensorflow.keras.models import load_model


# 2. Load model dan encoder
model = load_model("sistem_rekomendasi_model.h5")

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

with open("mlb.pkl", "rb") as f:
    mlb = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

# 3. Load dataset lokal yang sudah punya kolom 'mood'
df = pd.read_csv("dataset_fix.csv")
df.head()

# Proses ulang genres_list dan genre one-hot encoding (sesuai training)
df['genres_list'] = df['genres'].apply(lambda x: x.strip("[]").replace("'", "").split(", "))
genre_df = pd.DataFrame(mlb.transform(df['genres_list']), columns=mlb.classes_)

# Gabungkan fitur numerik dan genre one-hot
features = pd.concat([
    df[['release_year', 'popularity', 'rating']].reset_index(drop=True),
    genre_df.reset_index(drop=True)
], axis=1)

# Normalisasi
features_scaled = scaler.transform(features)

# 4. Fungsi rekomendasi berdasarkan input mood
def recommend_movies_by_mood(mood_input, top_k=5):
    mood_input = mood_input.capitalize()

    if mood_input not in le.classes_:
        return f"Mood '{mood_input}' tidak dikenali. Pilih salah satu dari: {list(le.classes_)}"

    # Cari indeks film yang sesuai dengan mood
    matching_indices = df[df['mood'] == mood_input].index

    if matching_indices.empty:
        return f"Tidak ada film yang cocok untuk mood '{mood_input}'."

    # Ambil subset fitur dan prediksi
    X_subset = features_scaled[matching_indices]
    preds = model.predict(X_subset)

    # Skor berdasarkan probabilitas kelas yang sesuai dengan input mood
    label_target = le.transform([mood_input])[0]
    match_score = preds[:, label_target]

    # Ambil top k berdasarkan skor tertinggi
    top_indices = match_score.argsort()[::-1][:top_k]
    result_indices = matching_indices[top_indices]

    return df.loc[result_indices][['title', 'genres', 'release_year', 'popularity', 'rating']]

# 5. Input dari user
user_mood = input("Masukkan mood Anda (contoh: 'Cheerful', 'Emotional', 'Energetic', 'Imaginative', 'Insightful', 'Romantic', 'Thrilling', 'Neutral' ): ")
top_k = input("Berapa banyak film yang ingin direkomendasikan? (Max = 5): ")

# Validasi top_k
try:
    top_k = int(top_k)
except:
    top_k = 5

# Tampilkan rekomendasi
rekomendasi = recommend_movies_by_mood(user_mood, top_k=top_k)

print("\n🎬 Rekomendasi Film:")
if isinstance(rekomendasi, pd.DataFrame) and not rekomendasi.empty:
    print(rekomendasi.to_string(index=False))
else:
    print("Tidak ada film yang cocok untuk filter yang dipilih.")

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(


Masukkan mood Anda (contoh: 'Cheerful', 'Emotional', 'Energetic', 'Imaginative', 'Insightful', 'Romantic', 'Thrilling', 'Neutral' ): Romantic
Berapa banyak film yang ingin direkomendasikan? (Max = 5): 4
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

🎬 Rekomendasi Film:
                          title                genres  release_year  popularity  rating
                     Sugar Baby  ['Romance', 'Drama']          2024     10.2960   7.200
             Fly Me to the Moon ['Romance', 'Comedy']          2024     10.7169   6.837
Confessions from a Holiday Camp ['Comedy', 'Romance']          1977      1.8051   5.278
                 Joseph Andrews ['Romance', 'Comedy']          1977      1.7131   4.700
